In [1]:
from typing import Any

from common.inputreader import InputReader, PuzzleWrapper

puzzle = PuzzleWrapper(year=int("2024"), day=int("24"))

puzzle.header()

# Crossed Wires

[Open Website](https://adventofcode.com/2024/day/24)

In [39]:
# helper functions
import re


class Connection:
    def __init__(self, inputs: tuple, operation: str, output: str):
        self.inputs = inputs
        self.operation = operation
        self.output = output
        self.degree = 0
        self.can_calculate = False

    def __str__(self):
        return f"{self.inputs} {self.operation} {self.output}"


def domain_from_input(input: InputReader) -> tuple:
    lines = input.lines_as_strs()

    wires = {}
    connections = []

    reading_registers = True
    for line in lines:
        if len(line) == 0:
            reading_registers = False
            continue

        if reading_registers:
            # trim last character of line[0]
            wire = line[0][:-1]
            wires[wire] = int(line[1])
        else:
            inputs = (line[0], line[2])
            operation = line[1]
            output = line[4]
            connection = Connection(inputs, operation, output)
            connections.append(connection)

    return wires, connections


test_input, test_connections = domain_from_input(puzzle.example(0))
print(test_input, test_connections)

{'x00': 1, 'x01': 1, 'x02': 1, 'y00': 0, 'y01': 1, 'y02': 0} [<__main__.Connection object at 0x10730fcb0>, <__main__.Connection object at 0x1079acf50>, <__main__.Connection object at 0x1079accd0>]


In [41]:
from matplotlib import pyplot as plt
# test case (part 1)
import networkx as nx


def draw_graph(G):
    pos = nx.spring_layout(G)
    edge_labels = nx.get_edge_attributes(G, 'label')
    nx.draw(G, pos, with_labels=True, node_size=2000, node_color='lightblue', font_size=10, font_weight='bold',
            arrows=True)
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='red')
    plt.show()


def calculate_result(wires):
    # find wires that start with z
    z_wires = [wire for wire in wires if wire.startswith("z")]
    # sort wires alphabetically
    z_wires = sorted(z_wires, reverse=True)
    z_wire_values = [wires[wire] for wire in z_wires]
    # concatenate values as a string and convert from binary to decimal
    result = int("".join([str(value) for value in z_wire_values]), 2)
    return result


def part_1(reader: InputReader, debug: bool) -> int:
    wires, connections = domain_from_input(reader)

    graph = nx.DiGraph()

    for connection in connections:
        graph.add_edge(connection.inputs[0], connection.output)
        graph.add_edge(connection.inputs[1], connection.output)

    # find the degree of each node
    in_degrees = graph.in_degree()

    for connection in connections:
        input_degrees = [in_degrees[input_wire] for input_wire in connection.inputs]
        connection.degree = max(input_degrees)

    # copy connections to queue
    queue = connections.copy()

    while len(queue) > 0:
        # mark connections that can be calculated
        for connection in queue:
            inputs = connection.inputs
            connection.can_calculate = all([input in wires for input in inputs])
        # sort by can_calculate
        queue.sort(key=lambda x: x.can_calculate, reverse=True)

        # calculate connection
        connection = queue.pop(0)


        inputs = connection.inputs
        input_values = [wires[input] for input in inputs]

        if connection.operation == "AND":
            wires[connection.output] = input_values[0] & input_values[1]
        elif connection.operation == "OR":
            wires[connection.output] = input_values[0] | input_values[1]
        elif connection.operation == "XOR":
            wires[connection.output] = input_values[0] ^ input_values[1]

    return calculate_result(wires)


result = part_1(puzzle.example(0), True)
print(result)
assert result == 4

result = part_1(puzzle.get_code_block(1), True)
print(result)
assert result == 2024

4
2024


In [42]:
# real case (part 1)
result = part_1(puzzle.input(), False)
print(result)

51715173446832


In [ ]:
# test case (part 2)
def part_2(reader: InputReader, debug: bool) -> int:
    lines = domain_from_input(reader)
    if debug:
        print(lines)
    return 0


result = part_2(puzzle.example(0), True)
print(result)
assert result == 0

In [ ]:
# real case (part 2)
result = part_2(puzzle.input(), False)
print(result)

In [ ]:
# print easters eggs
puzzle.print_easter_eggs()